In [2]:
kec_url = 'https://github.com/uluumbch/indonesia-locations-data/raw/refs/heads/main/final_results/kecamatan_lat_long.csv'
kota_kab_url = 'https://github.com/uluumbch/indonesia-locations-data/raw/refs/heads/main/final_results/kota_kab_lat_long.csv'
provinsi_url = 'https://github.com/uluumbch/indonesia-locations-data/raw/refs/heads/main/final_results/provinces_lat_long.csv'

In [3]:
!wget $kec_url -O kecamatan.csv
!wget $kota_kab_url -O kota_kab.csv
!wget $provinsi_url -O provinsi.csv

--2025-06-20 01:56:48--  https://github.com/uluumbch/indonesia-locations-data/raw/refs/heads/main/final_results/kecamatan_lat_long.csv
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/uluumbch/indonesia-locations-data/refs/heads/main/final_results/kecamatan_lat_long.csv [following]
--2025-06-20 01:56:48--  https://raw.githubusercontent.com/uluumbch/indonesia-locations-data/refs/heads/main/final_results/kecamatan_lat_long.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 420402 (411K) [text/plain]
Saving to: ‘kecamatan.csv’

kecamatan.csv       100%[===================>] 410.55K  --.-KB/s    in 

In [5]:
import pandas as pd
import os

# --- Configuration ---
# Define file paths
PROVINSI_FILE = 'provinsi.csv'
KOTA_KAB_FILE = 'kota_kab.csv'
KECAMATAN_FILE = 'kecamatan.csv'
OUTPUT_FILE = 'merged_indonesia_data.csv'

# --- Create Dummy Files if they don't exist ---
# This part is for demonstration purposes, so the script can run even without your files.
# You can remove this block if you have your actual CSV files in the same directory.
if not os.path.exists(PROVINSI_FILE):
    print(f"'{PROVINSI_FILE}' not found. Creating a dummy file for demonstration.")
    provinsi_data = {'id': [11], 'name': ['ACEH'], 'lat': [4.695135], 'long': [96.74939930000001]}
    pd.DataFrame(provinsi_data).to_csv(PROVINSI_FILE, index=False)

if not os.path.exists(KOTA_KAB_FILE):
    print(f"'{KOTA_KAB_FILE}' not found. Creating a dummy file for demonstration.")
    kota_kab_data = {'id': [1101], 'foreign': [11], 'name': ['KABUPATEN SIMEULUE'], 'lat': [2.583333], 'long': [96.083333]}
    pd.DataFrame(kota_kab_data).to_csv(KOTA_KAB_FILE, index=False)

if not os.path.exists(KECAMATAN_FILE):
    print(f"'{KECAMATAN_FILE}' not found. Creating a dummy file for demonstration.")
    kecamatan_data = {'id': [1101010], 'foreign': [1101], 'name': ['TEUPAH SELATAN'], 'lat': [2.4642851395543044], 'long': [96.52043027230327]}
    pd.DataFrame(kecamatan_data).to_csv(KECAMATAN_FILE, index=False)

In [13]:

# Step 1: Load the datasets with descriptive column names
# For provinsi, the 'id' is the key for other files to link to.
df_provinsi = pd.read_csv(PROVINSI_FILE)
df_provinsi = df_provinsi.iloc[:, 1:]
df_provinsi.columns = ['prov_id', 'prov_name', 'prov_lat', 'prov_long']

# For kota/kab, 'id' is its own key, and 'foreign' links to provinsi.
df_kota_kab = pd.read_csv(KOTA_KAB_FILE)
df_kota_kab = df_kota_kab.iloc[:, 1:]
df_kota_kab.columns = ['kota_kab_id', 'prov_id', 'kota_kab_name', 'kota_kab_lat', 'kota_kab_long']

# For kecamatan, 'id' is its own key, and 'foreign' links to kota/kab.
df_kecamatan = pd.read_csv(KECAMATAN_FILE)
df_kecamatan = df_kecamatan.iloc[:, 1:]
df_kecamatan.columns = ['kec_id', 'kota_kab_id', 'kec_name', 'kec_lat', 'kec_long']

print("--- Successfully loaded all CSV files. ---")

--- Successfully loaded all CSV files. ---


In [14]:

# Step 2: Merge Kecamatan with Kota/Kabupaten
# We use a 'left' merge to ensure we keep all kecamatan records.
# The 'on' parameter specifies the key to join on, which is 'kota_kab_id'.
# This connects each kecamatan to its respective kabupaten/kota.
merged_df = pd.merge(df_kecamatan, df_kota_kab, on='kota_kab_id', how='left')
print("--- Merged Kecamatan with Kota/Kabupaten. ---")


# Step 3: Merge the result with Provinsi
# Now, we take the already merged data and connect it to the provinsi data.
# The key for this merge is 'prov_id'.
final_df = pd.merge(merged_df, df_provinsi, on='prov_id', how='left')
print("--- Merged the result with Provinsi. ---")


# Step 4: Display and Save the final result
print("\n--- Final Merged DataFrame (First 5 Rows): ---")
print(final_df.head())

# Select and reorder columns for a clean final output
final_df = final_df[[
    'prov_id', 'prov_name',
    'kota_kab_id', 'kota_kab_name',
    'kec_id', 'kec_name',
    'kec_lat', 'kec_long',
    'kota_kab_lat', 'kota_kab_long',
    'prov_lat', 'prov_long'
]]

# Save the final merged dataframe to a new CSV file
final_df.to_csv(OUTPUT_FILE, index=False)
print(f"\n✅ Success! The combined data has been saved to '{OUTPUT_FILE}'")

--- Merged Kecamatan with Kota/Kabupaten. ---
--- Merged the result with Provinsi. ---

--- Final Merged DataFrame (First 5 Rows): ---
    kec_id  kota_kab_id         kec_name   kec_lat   kec_long  prov_id  \
0  1101010         1101   TEUPAH SELATAN  2.464285  96.520430       11   
1  1101020         1101   SIMEULUE TIMUR  2.640112  96.580326       11   
2  1101021         1101     TEUPAH BARAT  2.554213  96.349640       11   
3  1101022         1101    TEUPAH TENGAH  2.411233  96.334985       11   
4  1101030         1101  SIMEULUE TENGAH  2.645959  96.090894       11   

        kota_kab_name  kota_kab_lat  kota_kab_long prov_name  prov_lat  \
0  KABUPATEN SIMEULUE      2.583333      96.083333      ACEH  4.695135   
1  KABUPATEN SIMEULUE      2.583333      96.083333      ACEH  4.695135   
2  KABUPATEN SIMEULUE      2.583333      96.083333      ACEH  4.695135   
3  KABUPATEN SIMEULUE      2.583333      96.083333      ACEH  4.695135   
4  KABUPATEN SIMEULUE      2.583333      96.083333